# House Price Prediction

In [ ]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation

### Generate data

1. Generate housing data of sizes between 1000 and 3500 sq. ft.

In [ ]:
num_houses = 160
np.random.seed(42)
house_size = np.random.randint(low=1000, high=3500, size=num_houses)

2. Generate house prices from house size with a random noise added.

In [ ]:
np.random.seed(42)
house_price = house_size * 100.0 + np.random.randint(low=20000, high=70000, size=num_houses)

3. Visualize the house and price data

In [ ]:
plt.plot(house_size, house_price, "bo")
plt.ylabel("Price")
plt.xlabel("Size")
plt.show()

### Prepare training/testing data sets

4. Create function to normalize data values using standard normal distribution
5. Define number of training/testing samples, 70/30 split

In [ ]:
def normalize(array):
    return (array - array.mean()) / array.std()

In [ ]:
num_train_samples = math.floor(num_houses * 0.7)

# Training set
train_house_size = np.asarray(house_size[:num_train_samples])
train_house_price = np.asarray(house_price[:num_train_samples])

train_house_size_norm = normalize(train_house_size)
train_house_price_norm = normalize(train_house_price)

# Testing set
test_house_size = np.asarray(house_size[num_train_samples:])
test_house_price = np.asarray(house_price[num_train_samples:])

test_house_size_norm = normalize(test_house_price)
test_house_price_norm = normalize(test_house_price)

### Setup Tensorflow environment

6. Create Tensorflow placeholders that get updated as we descend through gradient descent

In [ ]:
tf_house_size = tf.placeholder("float", name="house_size")
tf_house_price = tf.placeholder("float", name="price")

7. Define variables holding the size factor and price offset we set during training

In [ ]:
tf_size_factor = tf.Variable(np.random.randn(), name="size_factor")
tf_price_offset = tf.Variable(np.random.randn(), name="price_offset")

8. Define the inference and loss functions for the predicting values - predicted price = (size_factor * house_size) + price_offset

In [ ]:
tf_price_pred = tf.add(tf.multiply(tf_size_factor, tf_house_size), tf_price_offset)
tf_cost = tf.reduce_sum(tf.pow(tf_price_pred - tf_house_price, 2)) / (2 * num_train_samples)

9. Define learning rate and optimizer

In [ ]:
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(tf_cost)

10. Initialize the variables and launch the graph in the session

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    # set how often to display training progress and number of training iterations
    display_every = 2
    num_training_iter = 50
    
    # Iterate through training data
    for iteration in range(num_training_iter):
        
        # Fit training data
        for(x,y) in zip(train_house_size_norm, train_house_price_norm):
            sess.run(optimizer, feed_dict={tf_house_size: x, tf_house_price: y})
            
        # Display current status
        if((iteration + 1) % display_every == 0 ):
            c = sess.run(tf_cost, feed_dict={tf_house_size: train_house_size_norm, tf_house_price: train_house_price_norm})
            print("Iteration #:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(c), \
                  "size_factor=", sess.run(tf_size_factor), "price_offset=", sess.run(tf_price_offset))
    print("Optimization Finished!")
    training_cost = sess.run(tf_cost, feed_dict={tf_house_size: train_house_size_norm, tf_house_price: train_house_price_norm})
    print("Trained cost=", training_cost, "size_factor=", sess.run(tf_size_factor), "price_offset=", sess.run(tf_price_offset), '\n')
    
    # Denormalize values to original scale
    train_house_size_mean = train_house_size.mean()
    train_house_size_std = train_house_size.std()

    train_house_price_mean = train_house_price.mean()
    train_house_price_std = train_house_price.std()

    # Plot the graph
    plt.rcParams["figure.figsize"] = (10,8)
    plt.figure()
    plt.ylabel("Price")
    plt.xlabel("Size (sq.ft.)")
    plt.plot(train_house_size, train_house_price, 'go', label="Training data")
    plt.plot(test_house_size, test_house_price, 'mo', label="Testing Data")
    plt.plot(train_house_size_norm * train_house_size_std + train_house_size_mean, (sess.run(tf_size_factor) * train_house_size_norm + sess.run(tf_price_offset)) * train_house_price_std + train_house_price_mean, label="Learned Regression")
    plt.legend(loc="upper left")
    plt.show()